The notebook is based on https://github.com/karpathy/nanoGPT and https://lena-voita.github.io/nlp_course.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -q

In [4]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print(f"length of dataset in characters: {len(text)}\n\n")
# let's look at the first 250 characters
print(text[:250])

length of dataset in characters: 1115394


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



**Tokenization**: 
Unlike the pictures we are used to in this course, the space of language texts is discrete, so a typical first step for an nlp task is to split raw data into characters, words or other tokens, this process is called tokenization.


<img src="https://lena-voita.github.io/resources/lectures/seq2seq/bpe/tokenization_word_subword-min.png" width=750/>

During the model inference we may come across tokens, which were not in the training vocabulary, than they will be replaced with a special UNK ("unknown") token. Therefore, if you use the straightforward word-level tokenization (i.e., your tokens are words), you will be able to process a fixed number of words. This is the fixed vocabulary problem : you will be getting lot's of unknown tokens, and your model won't translate them properly. 

But how can we represent all words, even those we haven't seen in the training data? Well, even if you are not familiar with a word, you are familiar with the parts it consists of - subwords (in the worst case, symbols). Then why don't we split the rare and unknown words into smaller parts?

This is exactly what was proposed in the paper [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf) by Rico Sennrich, Barry Haddow and Alexandra Birch. They introduced the de-facto standard subword segmentation, Byte Pair Encoding (BPE). BPE keeps frequent words intact and splits rare and unknown ones into smaller known parts. 

Basically you can trade-off the codebook size and the sqquence lengths, so you can have a very long sequence of integers with very small vocabulary or we can have shoer sequences of integers with very large vocabularies.
However, for the sake of simplicity in this notebook, we will focus on character-based tokenization

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("Vocab size: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size:  65


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:250]) # the 250 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

### The Simplest Seq2seq Model: Two RNNs for Encoder and Decoder

The simplest encoder-decoder model consists of two RNNs (LSTMs): one for the encoder and another for the decoder. Encoder RNN reads the source sentence, and the final state is used as the initial state of the decoder RNN. The hope is that the final encoder state "encodes" all information about the source, and the decoder can generate the target sentence based on this vector.

This model can have different modifications: for example, the encoder and decoder can have several layers. Such a model with several layers was used, for example, in the paper Sequence to Sequence Learning with Neural Networks - one of the first attempts to solve sequence-to-sequence tasks using neural networks. 

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/general/enc_dec_simple_rnn-min.png"/>

#### Training: The Cross-Entropy Loss
Seq2seq models are trained to predict probability distributions of the next token given previous context (source and previous target tokens). Intuitively, at each step we maximize the probability a model assigns to the correct token.

Formally, let's assume we have a training instance with the source $x=(x_1,...,x_m)$ and the target $y=(y_1,...,y_n)$. Then at the timestep $t$, a model predicts a probability distribution $p^{(t)}=p(*|y_1,...,y_{t-1},x_1,...,x_m)$. The target at this step is $p^* = onehot(y_t)$, i.e., we want a model to assign probability 1 to the correct token, $y_t$, and zero to the rest.

The standard loss function is the cross-entropy loss. Cross-entropy loss for the target distribution $p^*$ and the predicted distribution $p$ is
$$Loss(p^*, p)=-p^*\log(p) = - \sum_{i=1}^{|V|}p_{i}^{*}\log(p_i)$$

Since only one of $p_i^*$ is non-zero (for the correct token $y_t$), we will get
$$Loss(p^*, p)=-\log(p_{y_t}) = - \log(p(y_t|y_{<t},x))$$

At each step, we maximize the probability a model assigns to the correct token. Look at the illustration for a single timestep

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/general/one_step_loss_intuition-min.png"/>

#### Inference

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/general/inference_formula-min.png">

 - Greedy decoding: at each step, pick the most probable token
 - Beam Search: Keep track of several most probably hypotheses

In [9]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [10]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [11]:
get_batch('train')

(tensor([[63,  1, 40,  ..., 46,  1, 46],
         [56, 53, 61,  ..., 57, 57,  5],
         [52, 43, 60,  ..., 40, 50, 53],
         ...,
         [39, 50, 10,  ..., 50,  1, 49],
         [ 1, 51, 59,  ...,  0,  0, 28],
         [31, 13, 14,  ..., 59, 50, 58]], device='cuda:0'),
 tensor([[ 1, 40, 50,  ...,  1, 46, 47],
         [53, 61, 52,  ..., 57,  5, 42],
         [43, 60, 43,  ..., 50, 53, 53],
         ...,
         [50, 10,  0,  ...,  1, 49, 52],
         [51, 59, 57,  ...,  0, 28, 13],
         [13, 14, 17,  ..., 50, 58, 43]], device='cuda:0'))

In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Vanilla attention
<img src="https://lena-voita.github.io/resources/lectures/seq2seq/attention/general_scheme-min.png" width=600/>

### Self-Attention: the "Look at Each Other" Part
<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/decenc_vs_self-min.png" width=600/>

**Query, Key, and Value in Self-Attention**
- query - asking for information;
- key - saying that it has some information;
- value - giving the information.

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/qkv_explained-min.png" width=600/>
The formula for computing attention output is as follows:
<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/qkv_attention_formula-min.png" width=400/>

### Masked Self-Attention: "Don't Look Ahead" for the Decoder
In the decoder, self-attention is a bit different from the one in the encoder. While the encoder receives all tokens at once and the tokens can look at all tokens in the input sentence, in the decoder, we generate one token at a time: during generation, we don't know which tokens we'll generate in future.

To forbid the decoder to look ahead, the model uses masked self-attention: future tokens are masked out.

### Multi-Head Attention: Independently Focus on Different Things
Usually, understanding the role of a word in a sentence requires understanding how it is related to different parts of the sentence. This is important not only in processing source sentence but also in generating target. For example, in some languages, subjects define verb inflection (e.g., gender agreement), verbs define the case of their objects, and many more. What I'm trying to say is: each word is part of many relations.

Therefore, we have to let the model focus on different things: this is the motivation behind Multi-Head Attention. Instead of having one attention mechanism, multi-head attention has several "heads" which work independently. 

These heads play interpretable "roles".
These roles are:
   - positional: attend to a token's immediate neighbors, and the model has several such heads (usually 2-3 heads looking at the previous token and 2 heads looking at the next token);
   - syntactic: learned to track some major syntactic relations in the sentence (subject-verb, verb-object);
   - rare tokens: the most important head on the first layer attends to the least frequent tokens in a sentence (this is true for models trained on different language pairs!).
   

<table><tr>
<td><img src="https://lena-voita.github.io/resources/posts/acl19_heads/position_head/subs_en_ru_next-min.png" width=300/></td> 
<td><img src="https://lena-voita.github.io/resources/posts/acl19_heads/syntactic_head/subs_en_ru_sv_1-min.png" width=300/></td>
</tr></table>

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [14]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

### Transformer model architecture
<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/model-min.png">

In [15]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [16]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [17]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [18]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

10.788929 M parameters


In [19]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2480, val loss 4.2520
step 500: train loss 1.8106, val loss 1.9419
step 1000: train loss 1.4240, val loss 1.6381
step 1500: train loss 1.2860, val loss 1.5381
step 2000: train loss 1.2063, val loss 1.5047
step 2500: train loss 1.1339, val loss 1.4924
step 3000: train loss 1.0780, val loss 1.4866


KeyboardInterrupt: 

In [38]:
# generate from the model
context = torch.tensor(encode("Romeo: "), 
                       dtype=torch.long, device=device)[None,...]
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

Romeo: I pray thee, noise well: poor up
My sights have said, Juliet's bad: will I speak,
And stark by my simple blood from my here,
For all I humbly will not scorn thy short.

QUEEN MARGARET:
How our sons will do where abhorre thee!

QUEEN MARGARET:
They should not be thy wife's head, and worm too thee thorn.

QUEEN ELIZABETH:
Now, by the first of late grace to Angelo,
That you think his mistress' to be found.

KING HENRY VI:
Then, Hungerforward, his account I mean toward;
Give me leave the althing wit


In [43]:
# generate from the model
context = torch.tensor(encode("KING"), 
                       dtype=torch.long, device=device)[None,...]
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

KING EDWARD IV:
Yea, to be widow it to sooth get you at York
From your father, prophetest.

BUCKINGHAM:
My lord, come the Tower, immedy friends to have report
With your friends, more happing leads in judgment.

KING HENRY VI:
What, will we remember you.

GLOUCESTER:
Here comes well; of our kinsman is,
When the partners yet is gone and weak?

DERBY:
No, faint will mount to thy life;
Therefore, by the allowings government doth make Tarquis.

GLOUCESTER:
For, my lord, my lord. Your journeys shall prove


--------------------------